In [38]:
import ipywidgets as widgets
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
%matplotlib inline
import os
from ipywidgets import Layout

In [39]:
### datafile path  - change for your own
datafile ="tcosts_for map.csv"

In [40]:
def get_files(datafile):
    gdf = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))[['iso_a3', 'geometry']]
    raw = pd.read_csv(datafile)
    df = pd.pivot_table(raw, index = 'reporter', columns = 'year', values = 't_av').reset_index()
    df.sort_values(by='reporter')
    codes_change = {'ROM':'ROU',"TMP":"TLS","ZAR":"COD","PSE":"PSX","ANT":"SXM"}
    df['iso_a3'] = df['reporter'].map(codes_change)
    df['iso_a3'] = df['iso_a3'].fillna(df['reporter'])
    df = df.drop(columns = ['reporter'])
    merged = gdf.merge(df, on='iso_a3',indicator=True)
    return merged

merged = get_files(datafile)

year_list = ['1995', '1996', '1997', '1998', '1999', '2000', '2001','2002', '2003', '2004',
             '2005','2006', '2007', '2008', '2009', '2010','2011', '2012', '2013', '2014']

In [2]:
#!jupyter nbextension enable --py widgetsnbextension --sys-prefix

In [7]:
a = widgets.Dropdown(
    options=year_list,
    value='2014',
    description='Year:',
    disabled=False,
    layout=Layout(width="75%", height='30px'))

def f(a):
    fig, ax = plt.subplots(1, 1, figsize=(30, 30),constrained_layout=True)
    divider = make_axes_locatable(ax)
    cax = divider.append_axes("right", size="5%", pad=0.1)
    merged.plot(column=a,
            missing_kwds={'color': 'lightgrey'},
            ax=ax, 
            legend=True,
            cax=cax, 
            cmap='inferno_r')
    ax.set_title('{} Trade Costs'.format(a), fontdict={'fontsize': 30}, loc='center')
    ax.set_axis_off()

out = widgets.interactive_output(f, {'a': a})

widgets.VBox([widgets.VBox([a]), out])

